In [1]:
# linear algebra 
import numpy as np 
# data processing, CSV file I / O (e.g. pd.read_csv) 
import pandas as pd 
from tensorflow import keras
import os 
import tensorflow as tf 
from keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from keras.models import Model 
from keras.layers import Flatten, Dense, LSTM, Dropout, Embedding, Activation 
from keras.layers import concatenate, BatchNormalization, Input
from keras.layers import add 
from keras.utils import to_categorical, plot_model 
from keras.applications.inception_v3 import InceptionV3, preprocess_input 
import matplotlib.pyplot as plt # for plotting data 
import cv2

2024-07-09 08:51:46.622199: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 08:51:46.622316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 08:51:46.762686: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
reports_path = "/kaggle/input/chest-xrays-indiana-university/indiana_reports.csv"
reports_df = pd.read_csv(reports_path)
reports_df.head()

,uid,MeSH,Problems,image,indication,comparison,findings,impression
0,1,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
1,2,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
2,3,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,NaN,"No displaced rib fractures, pneumothorax, or p..."
3,4,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...
4,5,Osteophyte/thoracic vertebrae/multiple/small;T...,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.


In [3]:
projections_path="/kaggle/input/chest-xrays-indiana-university/indiana_projections.csv"
projections_df = pd.read_csv(projections_path)
projections_df.head()

,uid,filename,projection
0,1,1_IM-0001-4001.dcm.png,Frontal
1,1,1_IM-0001-3001.dcm.png,Lateral
2,2,2_IM-0652-1001.dcm.png,Frontal
3,2,2_IM-0652-2001.dcm.png,Lateral
4,3,3_IM-1384-1001.dcm.png,Frontal


In [4]:
merged_df = pd.merge(projections_df ,reports_df , on ="uid" )

In [5]:
merged_df

,uid,filename,projection,MeSH,Problems,image,indication,comparison,findings,impression
0,1,1_IM-0001-4001.dcm.png,Frontal,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
1,1,1_IM-0001-3001.dcm.png,Lateral,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
2,2,2_IM-0652-1001.dcm.png,Frontal,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
3,2,2_IM-0652-2001.dcm.png,Lateral,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
4,3,3_IM-1384-1001.dcm.png,Frontal,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,NaN,"No displaced rib fractures, pneumothorax, or p..."
...,...,...,...,...,...,...,...,...,...,...
7461,3997,3997_IM-2048-1002.dcm.png,Lateral,Opacity/lung/upper lobe/right/round/small;Gran...,Opacity;Granuloma,PA and lateral views of the chest.,XXXX-year-old male with positive PPD.,None available.,"Heart size within normal limits. Small, nodula...","No acute findings, no evidence for active TB."
7462,3998,3998_IM-2048-1001.dcm.png,Frontal,normal,normal,"PA and lateral chest XXXX, XXXX XXXX comparis...",tuberculosis positive PPD,NaN,NaN,Heart size is normal and the lungs are clear.
7463,3998,3998_IM-2048-1002.dcm.png,Lateral,normal,normal,"PA and lateral chest XXXX, XXXX XXXX comparis...",tuberculosis positive PPD,NaN,NaN,Heart size is normal and the lungs are clear.
7464,3999,3999_IM-2049-1001.dcm.png,Frontal,normal,normal,"CHEST PA and LATERAL: on XXXX, XXXX.",This is a XXXX-year-old female patient with sh...,"Chest x-XXXX, XXXX, XXXX.",NaN,The cardiac silhouette is normal in size and c...


In [6]:
final_df = merged_df[merged_df['projection'] != 'Lateral' ]

In [7]:
final_df

,uid,filename,projection,MeSH,Problems,image,indication,comparison,findings,impression
0,1,1_IM-0001-4001.dcm.png,Frontal,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
2,2,2_IM-0652-1001.dcm.png,Frontal,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
4,3,3_IM-1384-1001.dcm.png,Frontal,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,NaN,"No displaced rib fractures, pneumothorax, or p..."
6,4,4_IM-2050-1001.dcm.png,Frontal,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...
8,5,5_IM-2117-1003002.dcm.png,Frontal,Osteophyte/thoracic vertebrae/multiple/small;T...,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.
...,...,...,...,...,...,...,...,...,...,...
7456,3995,3995_IM-2046-1001.dcm.png,Frontal,Lung/hyperdistention/mild;Diaphragm/bilateral/...,Lung;Diaphragm;Cicatrix;Pulmonary Atelectasis;...,Xray Chest PA and Lateral,"Nausea, vomiting x2 weeks. Dialysis patient.","XXXX, XXXX.",The cardiomediastinal silhouette and pulmonary...,1. Interval resolution of bibasilar airspace d...
7458,3996,3996_IM-2047-1001.dcm.png,Frontal,Spine/degenerative,Spine,Xray Chest PA and Lateral,NaN,None.,The lungs are clear. Heart size is normal. No ...,Clear lungs. No acute cardiopulmonary abnormal...
7460,3997,3997_IM-2048-1001.dcm.png,Frontal,Opacity/lung/upper lobe/right/round/small;Gran...,Opacity;Granuloma,PA and lateral views of the chest.,XXXX-year-old male with positive PPD.,None available.,"Heart size within normal limits. Small, nodula...","No acute findings, no evidence for active TB."
7462,3998,3998_IM-2048-1001.dcm.png,Frontal,normal,normal,"PA and lateral chest XXXX, XXXX XXXX comparis...",tuberculosis positive PPD,NaN,NaN,Heart size is normal and the lungs are clear.


In [8]:
result_dict = dict(zip(final_df['filename'], final_df['impression']))

In [9]:
result_dict

{'1_IM-0001-4001.dcm.png': 'Normal chest x-XXXX.',
 '2_IM-0652-1001.dcm.png': 'No acute pulmonary findings.',
 '3_IM-1384-1001.dcm.png': 'No displaced rib fractures, pneumothorax, or pleural effusion identified. Well-expanded and clear lungs. Mediastinal contour within normal limits. No acute cardiopulmonary abnormality identified.',
 '4_IM-2050-1001.dcm.png': '1. Bullous emphysema and interstitial fibrosis. 2. Probably scarring in the left apex, although difficult to exclude a cavitary lesion. 3. Opacities in the bilateral upper lobes could represent scarring, however the absence of comparison exam, recommend short interval followup radiograph or CT thorax to document resolution.',
 '5_IM-2117-1003002.dcm.png': 'No acute cardiopulmonary abnormality.',
 '6_IM-2192-1001.dcm.png': 'No acute cardiopulmonary findings.',
 '7_IM-2263-1001.dcm.png': 'Basilar atelectasis. No confluent lobar consolidation or pleural effusion.',
 '8_IM-2333-1001.dcm.png': 'No acute cardiopulmonary disease.',
 '9

In [10]:
import string
def clean_captions(captions): 
    keys_to_remove = []
    for key, caption in captions.items():
        if  isinstance(caption, str): 
            caption = [ch for ch in caption if ch not in string.punctuation] 
            caption = ''.join(caption) 
            caption = caption.split(' ') 
            caption = [word.lower() for word in caption if len(word)>1 and word.isalpha()] 
            caption = ' '.join(caption) 
        else:
            keys_to_remove.append(key)
    for key in keys_to_remove:
        del captions[key]
    return captions
result_dict = clean_captions(result_dict) 
result_dict

{'1_IM-0001-4001.dcm.png': 'Normal chest x-XXXX.',
 '2_IM-0652-1001.dcm.png': 'No acute pulmonary findings.',
 '3_IM-1384-1001.dcm.png': 'No displaced rib fractures, pneumothorax, or pleural effusion identified. Well-expanded and clear lungs. Mediastinal contour within normal limits. No acute cardiopulmonary abnormality identified.',
 '4_IM-2050-1001.dcm.png': '1. Bullous emphysema and interstitial fibrosis. 2. Probably scarring in the left apex, although difficult to exclude a cavitary lesion. 3. Opacities in the bilateral upper lobes could represent scarring, however the absence of comparison exam, recommend short interval followup radiograph or CT thorax to document resolution.',
 '5_IM-2117-1003002.dcm.png': 'No acute cardiopulmonary abnormality.',
 '6_IM-2192-1001.dcm.png': 'No acute cardiopulmonary findings.',
 '7_IM-2263-1001.dcm.png': 'Basilar atelectasis. No confluent lobar consolidation or pleural effusion.',
 '8_IM-2333-1001.dcm.png': 'No acute cardiopulmonary disease.',
 '9

In [11]:
def to_vocab(captions): 
    words = set() 
    for key,caption in captions.items():  
        words.update(caption.split()) 
    return words 
vocab = to_vocab(result_dict)
vocab

{'humeral',
 'procedure.',
 'cleared',
 'tracheostomy',
 'artery.',
 'elevated,',
 'clavicle',
 'Basilar',
 'evident',
 'junction',
 'may',
 'Clips',
 'overlie',
 'pathology;',
 'helpful',
 'joints.',
 'Rotated',
 'resolved',
 'entirely',
 'characterized',
 'valve',
 'bullets',
 'aorta.',
 'supporting',
 'cm.',
 'shows',
 'Pelvis.',
 'disruption',
 'Indistinct',
 'element',
 'consideration,',
 'postsurgical',
 'zone',
 'stable/unremarkable',
 'slightly',
 'consider',
 'communicated',
 'effusions,',
 'surgery',
 'emphysema',
 'abdomen',
 'Irregularity',
 'Chest',
 'Anticipated',
 'arteries.',
 'tendinitis.',
 'attention',
 'indicated',
 'neoplasm.',
 'spondylosis.',
 'Clavicle.',
 'hyper',
 'message',
 'represent',
 'atelectasis.',
 'prominence,',
 'only.',
 'scan.',
 '1',
 'secondary',
 'With',
 'reflects',
 'worrisome.',
 'calcification,',
 'Blunting',
 'collections.',
 'Tuberculosis',
 'muscular',
 'potential',
 'joint',
 'placement.',
 'medication',
 'accepted',
 'Bony',
 'fibrosis'

In [12]:
len(vocab)

2521

In [13]:
len(result_dict.keys())

3788

In [14]:
images_path = '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/'
img_paths = []
for key in result_dict.keys():
    img_paths.append(images_path + key)
img_paths
print(len(img_paths))


3788


In [15]:
img_paths

['/kaggle/input/chest-xrays-indiana-university/images/images_normalized/1_IM-0001-4001.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/2_IM-0652-1001.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/3_IM-1384-1001.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/4_IM-2050-1001.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/5_IM-2117-1003002.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/6_IM-2192-1001.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/7_IM-2263-1001.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/8_IM-2333-1001.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/9_IM-2407-1001.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/10_IM-0002-2001.dcm.png',
 '/kaggle/input/chest-xray

In [16]:
train_images_paths = img_paths[:3000]
val_images_paths = img_paths[3000:3400]
test_images_paths = img_paths[3400:3788]
len(train_images_paths), len(val_images_paths), len(test_images_paths)

(3000, 400, 388)

In [17]:
test_images_paths

['/kaggle/input/chest-xrays-indiana-university/images/images_normalized/3599_IM-1775-1001.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/3600_IM-1776-4004.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/3601_IM-1777-2002.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/3602_IM-1778-1001.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/3603_IM-1779-1001.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/3604_IM-1780-0001-0001.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/3605_IM-1781-1001.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/3606_IM-1781-1001.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/3607_IM-1781-1001.dcm.png',
 '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/3608_IM-1781-1001.dcm.p

In [18]:
train_img_names = [ imgname[len(images_path):] for imgname in train_images_paths]
train_img_names

['1_IM-0001-4001.dcm.png',
 '2_IM-0652-1001.dcm.png',
 '3_IM-1384-1001.dcm.png',
 '4_IM-2050-1001.dcm.png',
 '5_IM-2117-1003002.dcm.png',
 '6_IM-2192-1001.dcm.png',
 '7_IM-2263-1001.dcm.png',
 '8_IM-2333-1001.dcm.png',
 '9_IM-2407-1001.dcm.png',
 '10_IM-0002-2001.dcm.png',
 '11_IM-0067-1001.dcm.png',
 '12_IM-0133-1001.dcm.png',
 '13_IM-0198-1001.dcm.png',
 '14_IM-0256-1001.dcm.png',
 '15_IM-0324-1001.dcm.png',
 '17_IM-0460-2001.dcm.png',
 '18_IM-0520-1001.dcm.png',
 '19_IM-0583-1001.dcm.png',
 '20_IM-0653-1001.dcm.png',
 '21_IM-0729-1001-0001.dcm.png',
 '22_IM-0810-1001.dcm.png',
 '23_IM-0879-1001.dcm.png',
 '24_IM-0949-1001.dcm.png',
 '25_IM-1024-2001.dcm.png',
 '26_IM-1090-1001.dcm.png',
 '27_IM-1168-2002.dcm.png',
 '28_IM-1231-1001.dcm.png',
 '29_IM-1302-1001.dcm.png',
 '30_IM-1385-1001.dcm.png',
 '31_IM-1450-1001.dcm.png',
 '32_IM-1511-1001.dcm.png',
 '33_IM-1576-13013.dcm.png',
 '34_IM-1644-1001.dcm.png',
 '35_IM-1704-1001.dcm.png',
 '36_IM-1776-1001.dcm.png',
 '37_IM-1847-0001-00

In [19]:
len(train_img_names)

3000

In [20]:
# result_dict

In [21]:
def load_clean_captions(captions, dataset): 
    dataset_des = dict() 
    for key, caption in captions.items(): 
        if key in dataset: 
            dataset_des[key]= 'startseq ' + caption + ' endseq'
    return dataset_des 

train_descriptions = load_clean_captions(result_dict, train_img_names) 
# print(train_descriptions['1000268201_693b08cb0e'])
len(train_descriptions)
# train_descriptions

3000

In [22]:
base_model = InceptionV3(weights = 'imagenet') 
cnnmodel = Model(base_model.input, base_model.layers[-2].output)

96112376/96112376 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [24]:
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
def preprocess_img(img_path): 
	# inception v3 excepts img in 299 * 299 * 3 
	img = load_img(img_path, target_size = (299, 299)) # img = PIL image object 
	x = img_to_array(img) # x = img_to_array(img)  

	# Add one more dimension 
	x = np.expand_dims(x, axis = 0) #(1,299,299,3)
	x = preprocess_input(x) # normalization, resizing
	return x 

def encode(image): 
	image = preprocess_img(image) 
	vec = cnnmodel.predict(image) 
	vec = np.reshape(vec, (vec.shape[1])) #  if (1, N), it reshapes it to (N,).
	return vec 

In [25]:
encoding_train = {} 
print(len(train_images_paths))
i=1
for img in train_images_paths:
    print(i)
    i+=1
    encoding_train[img[len(images_path):]] = encode(img)

3000
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


I0000 00:00:1720515296.701657     115 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
17
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
22
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
23
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
24
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
26
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
27
1/1 ━━━━━━━━━━━━━━━━

In [26]:
encoding_test ={}
i=1
for img in test_images_paths:
    print(i)
    i+=1
    encoding_test[img[len(images_path):]] = encode(img)

1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
17
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
22
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
23
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
24
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
25
1/1 ━━━━━━━━━━━━━━━━━━

In [27]:
encoding_val ={}
i=1
for img in val_images_paths:
    print(i)
    i+=1
    encoding_val[img[len(images_path):]] = encode(img)

1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
17
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
22
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
23
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
24
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
25
1/1 ━━━━━━━━━━━━━━━━━━

In [28]:
# list of all training captions 
all_train_captions = [] 
for key, val in train_descriptions.items(): 
    all_train_captions.append(val) 
        
# consider only words which occur atleast 10 times 
vocabulary = vocab 
threshold = 10 # you can change this value according to your need 
word_counts = {} 
for cap in all_train_captions: 
	for word in cap.split(' '): 
		word_counts[word] = word_counts.get(word, 0) + 1
vocab = [word for word in word_counts if word_counts[word] >= threshold] 
len(vocab)
# word mapping to integers 
ixtoword = {} 
wordtoix = {} 

ix = 1
for word in vocab: 
	wordtoix[word] = ix 
	ixtoword[ix] = word 
	ix += 1
len(ixtoword) == len(wordtoix)
# find the maximum length of a description in a dataset 
# max_length = max(len(des.split()) for des in all_train_captions) 
# max_length

True

In [29]:
import json
with open('ixtoword.json', 'w') as f:
    json.dump(ixtoword, f)

In [30]:
import json
with open('wordtoix.json', 'w') as f:
    json.dump(wordtoix, f)

In [31]:
word_counts

{'startseq': 3000,
 'Normal': 104,
 'chest': 190,
 'x-XXXX.': 32,
 'endseq': 3000,
 'No': 2268,
 'acute': 1817,
 'pulmonary': 360,
 'findings.': 246,
 'displaced': 10,
 'rib': 40,
 'fractures,': 3,
 'pneumothorax,': 3,
 'or': 308,
 'pleural': 262,
 'effusion': 104,
 'identified.': 92,
 'Well-expanded': 35,
 'and': 440,
 'clear': 77,
 'lungs.': 130,
 'Mediastinal': 60,
 'contour': 52,
 'within': 118,
 'normal': 163,
 'limits.': 78,
 'cardiopulmonary': 1275,
 'abnormality': 94,
 '1.': 607,
 'Bullous': 2,
 'emphysema': 7,
 'interstitial': 84,
 'fibrosis.': 9,
 '2.': 293,
 'Probably': 1,
 'scarring': 40,
 'in': 227,
 'the': 605,
 'left': 293,
 'apex,': 2,
 'although': 3,
 'difficult': 5,
 'to': 202,
 'exclude': 9,
 'a': 90,
 'cavitary': 3,
 'lesion.': 1,
 '3.': 47,
 'Opacities': 1,
 'bilateral': 70,
 'upper': 124,
 'lobes': 3,
 'could': 42,
 'represent': 76,
 'scarring,': 8,
 'however': 7,
 'absence': 2,
 'of': 736,
 'comparison': 28,
 'exam,': 3,
 'recommend': 11,
 'short': 1,
 'interval'

In [32]:
all_train_captions

['startseq Normal chest x-XXXX. endseq',
 'startseq No acute pulmonary findings. endseq',
 'startseq No displaced rib fractures, pneumothorax, or pleural effusion identified. Well-expanded and clear lungs. Mediastinal contour within normal limits. No acute cardiopulmonary abnormality identified. endseq',
 'startseq 1. Bullous emphysema and interstitial fibrosis. 2. Probably scarring in the left apex, although difficult to exclude a cavitary lesion. 3. Opacities in the bilateral upper lobes could represent scarring, however the absence of comparison exam, recommend short interval followup radiograph or CT thorax to document resolution. endseq',
 'startseq No acute cardiopulmonary abnormality. endseq',
 'startseq No acute cardiopulmonary findings. endseq',
 'startseq Basilar atelectasis. No confluent lobar consolidation or pleural effusion. endseq',
 'startseq No acute cardiopulmonary disease. endseq',
 'startseq Increased size of density in the left cardiophrenic XXXX. Primary different

In [33]:
len(wordtoix)

407

In [34]:
train_features = encoding_train

In [35]:
max_length = 40

In [36]:
X1, X2, y = list(), list(), list() 
for key, caption in train_descriptions.items(): 
    pic = train_features[key] 
    # sequence of indices(corresponding integer index ) of a caption
    seq = [wordtoix[word] for word in caption.split(' ') if word in wordtoix]  
    for i in range(1, len(seq)): 
        # in_seq is the input sequence up to the i-th word.
        #out_seq is the i-th word, which is the next word in the sequence.
        in_seq, out_seq = seq[:i], seq[i] 
        in_seq = pad_sequences([in_seq], maxlen = max_length)[0] 
        out_seq = to_categorical([out_seq], num_classes = len(wordtoix) +1)[0] 
        # store 
        X1.append(pic)
        X2.append(in_seq) 
        y.append(out_seq) 
  
X2 = np.array(X2) 
X1 = np.array(X1) 
y = np.array(y) 

In [37]:
# Import necessary libraries
from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences
import numpy as np

# Prepare the training data for Word2Vec
sentences = [caption.split() for caption in result_dict.values()]

# Train the Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=200, window=5, min_count=1, workers=4)

# Get the word vectors and vocabulary
word_vectors = word2vec_model.wv
vocab = word_vectors.key_to_index

# Create the embedding matrix
emb_dim = 200
embedding_matrix = np.zeros((len(wordtoix) + 1, emb_dim))
for word, i in wordtoix.items():
    if word in word_vectors:
        embedding_matrix[i] = word_vectors[word]
# embeddings_index = {} 
# golve_path ='/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.200d.txt'
# #  reads the entire contents of the file into a string.
# glove = open(golve_path, 'r', encoding = 'utf-8').read() 
# for line in glove.split("\n"): 
#     values = line.split(" ") 
#     word = values[0] 
#     indices = np.asarray(values[1: ], dtype = 'float32') 
#     embeddings_index[word] = indices 
  
# emb_dim = 200
# emb_matrix = np.zeros((len(wordtoix) +1, emb_dim)) 
# for word, i in wordtoix.items(): 
#     emb_vec = embeddings_index.get(word) 
#     if emb_vec is not None: 
#         emb_matrix[i] = emb_vec 
# emb_matrix.shape

In [38]:
embedding_matrix[:10]

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.17190118, -0.03639833, -0.1015681 , ..., -0.10522034,
        -0.15426302, -0.02812929],
       ...,
       [ 0.01493348, -0.07999836, -0.14819852, ..., -0.25506932,
        -0.1305069 , -0.19113865],
       [ 0.18622981, -0.05593376, -0.11324337, ..., -0.16229504,
        -0.20045058, -0.06461325],
       [ 0.04507773, -0.04450602, -0.06890741, ..., -0.11141858,
        -0.08734099, -0.06901202]])

In [39]:
val_img_names = [ imgname[len(images_path):] for imgname in val_images_paths]
val_img_names

['3188_IM-1504-1001.dcm.png',
 '3189_IM-1505-1001.dcm.png',
 '3190_IM-1505-1001.dcm.png',
 '3191_IM-1505-1001.dcm.png',
 '3192_IM-1505-1001.dcm.png',
 '3193_IM-1505-1001.dcm.png',
 '3194_IM-1505-1001.dcm.png',
 '3195_IM-1506-0001-0002.dcm.png',
 '3196_IM-1507-1001.dcm.png',
 '3197_IM-1508-1001.dcm.png',
 '3198_IM-1509-4004.dcm.png',
 '3199_IM-1510-1001.dcm.png',
 '3200_IM-1512-1001.dcm.png',
 '3201_IM-1513-1001.dcm.png',
 '3202_IM-1513-1001.dcm.png',
 '3203_IM-1513-1001.dcm.png',
 '3204_IM-1513-1001.dcm.png',
 '3205_IM-1513-1001.dcm.png',
 '3206_IM-1513-1001.dcm.png',
 '3207_IM-1514-1001.dcm.png',
 '3208_IM-1515-1001.dcm.png',
 '3209_IM-1515-1001.dcm.png',
 '3210_IM-1517-1001.dcm.png',
 '3211_IM-1517-1001-0002.dcm.png',
 '3212_IM-1517-1001.dcm.png',
 '3213_IM-1518-3001.dcm.png',
 '3214_IM-1519-1001.dcm.png',
 '3215_IM-1520-1002.dcm.png',
 '3216_IM-1520-1001.dcm.png',
 '3218_IM-1520-1001.dcm.png',
 '3218_IM-1520-3001.dcm.png',
 '3219_IM-1520-1001.dcm.png',
 '3220_IM-1522-1001.dcm.png',


In [40]:
val_descriptions = load_clean_captions(result_dict, val_img_names) 
# print(train_descriptions['1000268201_693b08cb0e'])
len(val_descriptions)

400

In [41]:
val_features = encoding_val

In [42]:
X1_val, X2_val, y_val = list(), list(), list() 
for key, caption in val_descriptions.items(): 
    pic = val_features[key] 
    seq = [wordtoix[word] for word in caption.split(' ') if word in wordtoix] 
    for i in range(1, len(seq)): 
        in_seq, out_seq = seq[:i], seq[i] 
        in_seq = pad_sequences([in_seq], maxlen = max_length)[0] 
        out_seq = to_categorical([out_seq], num_classes = len(wordtoix) + 1)[0] 
        # store 
        X1_val.append(pic) 
        X2_val.append(in_seq) 
        y_val.append(out_seq) 
  
X2_val = np.array(X2_val) 
X1_val = np.array(X1_val) 
y_val = np.array(y_val) 

In [43]:
encoding_train.keys()

dict_keys(['1_IM-0001-4001.dcm.png', '2_IM-0652-1001.dcm.png', '3_IM-1384-1001.dcm.png', '4_IM-2050-1001.dcm.png', '5_IM-2117-1003002.dcm.png', '6_IM-2192-1001.dcm.png', '7_IM-2263-1001.dcm.png', '8_IM-2333-1001.dcm.png', '9_IM-2407-1001.dcm.png', '10_IM-0002-2001.dcm.png', '11_IM-0067-1001.dcm.png', '12_IM-0133-1001.dcm.png', '13_IM-0198-1001.dcm.png', '14_IM-0256-1001.dcm.png', '15_IM-0324-1001.dcm.png', '17_IM-0460-2001.dcm.png', '18_IM-0520-1001.dcm.png', '19_IM-0583-1001.dcm.png', '20_IM-0653-1001.dcm.png', '21_IM-0729-1001-0001.dcm.png', '22_IM-0810-1001.dcm.png', '23_IM-0879-1001.dcm.png', '24_IM-0949-1001.dcm.png', '25_IM-1024-2001.dcm.png', '26_IM-1090-1001.dcm.png', '27_IM-1168-2002.dcm.png', '28_IM-1231-1001.dcm.png', '29_IM-1302-1001.dcm.png', '30_IM-1385-1001.dcm.png', '31_IM-1450-1001.dcm.png', '32_IM-1511-1001.dcm.png', '33_IM-1576-13013.dcm.png', '34_IM-1644-1001.dcm.png', '35_IM-1704-1001.dcm.png', '36_IM-1776-1001.dcm.png', '37_IM-1847-0001-0002.dcm.png', '38_IM-1911-

In [44]:
np.size(encoding_train['1_IM-0001-4001.dcm.png'])

2048

In [45]:
validation_data = ([X1_val, X2_val], y_val)

In [46]:
vocab_size = len(wordtoix) + 1

In [47]:
# define the model 
ip1 = Input(shape = (2048, )) 
ip2 = Input(shape = (max_length, )) 

fe1 = Dropout(0.2)(ip1) 
fe2 = Dense(256, activation = 'relu')(fe1) 

se1 = Embedding(vocab_size, emb_dim, mask_zero = True)(ip2) 
se2 = Dropout(0.2)(se1) 
se3 = LSTM(256, use_cudnn=False)(se2) 

decoder1 = add([fe2, se3]) 
decoder2 = Dense(256, activation = 'relu')(decoder1) 

outputs = Dense(vocab_size, activation = 'softmax')(decoder2) 
model = Model(inputs = [ip1, ip2], outputs = outputs)

In [48]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [49]:
model.layers[2].set_weights([embedding_matrix]) 
model.layers[2].trainable = True
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"]) 
model.fit([X1, X2], y, epochs = 50, batch_size = 64,validation_data=validation_data)

Epoch 1/50
 15/475 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.0543 - loss: 5.5945   

W0000 00:00:1720516662.801687     116 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


475/475 ━━━━━━━━━━━━━━━━━━━━ 18s 24ms/step - accuracy: 0.2153 - loss: 4.3197 - val_accuracy: 0.3703 - val_loss: 3.1644
Epoch 2/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.3726 - loss: 3.0891 - val_accuracy: 0.4278 - val_loss: 2.7442
Epoch 3/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.4357 - loss: 2.6431 - val_accuracy: 0.4597 - val_loss: 2.5516
Epoch 4/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.4705 - loss: 2.3826 - val_accuracy: 0.4683 - val_loss: 2.4862
Epoch 5/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.4913 - loss: 2.2042 - val_accuracy: 0.4737 - val_loss: 2.4738
Epoch 6/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.5131 - loss: 2.0368 - val_accuracy: 0.4876 - val_loss: 2.4573
Epoch 7/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.5333 - loss: 1.9030 - val_accuracy: 0.4844 - val_loss: 2.4816
Epoch 8/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.5545 - loss: 1.7589 - val_accuracy: 0.48

In [51]:
model.save("indianamodel.keras")

In [52]:
def greedy_search2(pic):
    start = 'startseq'
    for i in range(max_length):
        seq = [wordtoix[word] for word in start.split() if word in wordtoix]
        seq = pad_sequences([seq], maxlen=max_length)
        yhat = model.predict([pic, seq])
        yhat = np.argmax(yhat,axis=-1)
        word = ixtoword[yhat[0]]
        start += ' ' + word
        if word == 'endseq':
            break
    final = start.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [53]:
encoding_test.keys()

dict_keys(['3599_IM-1775-1001.dcm.png', '3600_IM-1776-4004.dcm.png', '3601_IM-1777-2002.dcm.png', '3602_IM-1778-1001.dcm.png', '3603_IM-1779-1001.dcm.png', '3604_IM-1780-0001-0001.dcm.png', '3605_IM-1781-1001.dcm.png', '3606_IM-1781-1001.dcm.png', '3607_IM-1781-1001.dcm.png', '3608_IM-1781-1001.dcm.png', '3609_IM-1782-2001.dcm.png', '3610_IM-1783-1001.dcm.png', '3611_IM-1784-0001-0001.dcm.png', '3612_IM-1785-1001.dcm.png', '3613_IM-1786-2002.dcm.png', '3615_IM-1788-1001.dcm.png', '3616_IM-1789-2002.dcm.png', '3617_IM-1789-1001.dcm.png', '3618_IM-1790-82420001.dcm.png', '3619_IM-1791-1001.dcm.png', '3620_IM-1791-1001-0001.dcm.png', '3621_IM-1792-3001.dcm.png', '3623_IM-1793-1001.dcm.png', '3624_IM-1794-1001.dcm.png', '3625_IM-1794-1001.dcm.png', '3626_IM-1794-1001.dcm.png', '3628_IM-1796-1001.dcm.png', '3629_IM-1797-1001.dcm.png', '3630_IM-1798-4004.dcm.png', '3631_IM-1798-4004.dcm.png', '3632_IM-1799-2001.dcm.png', '3633_IM-1800-1001.dcm.png', '3634_IM-1801-1001.dcm.png', '3635_IM-1802

In [54]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


# for key,val in encoding_test.items():
#     print(val.shape)
image_p = '3671_IM-1827-3001.dcm.png'
image_features = encoding_test[image_p].reshape(1,2048)
caption = greedy_search2(image_features)  # Generate caption using greedy search
print("Generated Caption:", caption)
# Assuming 'image_path' contains the path to the image file
image_path = images_path + image_p
print(result_dict[image_p])
# Load the image using Matplotlib
img = mpimg.imread(image_path) 
# print("jhdkshfj",caption)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 573ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Generated Caption: No acute cardiopulmonary abnormality. .
1. Persistent mild right upper lobe infiltrate, similar to slightly improved from XXXX. 2. Left lung grossly clear.


In [56]:
model.save("modelAnup.h5")

In [58]:
model.save("modelAnup.keras")

In [59]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np

# Ensure you have NLTK's necessary resources
nltk.download('punkt')

# Define a function to calculate BLEU score
def calculate_bleu_score(references, candidate):
    smoothie = SmoothingFunction().method4
    return sentence_bleu(references, candidate, smoothing_function=smoothie)

# To calculate BLEU score for your dataset, you need to iterate over all test images and their captions:
def get_bleu_scores(encoding_test, result_dict, model, wordtoix, ixtoword, max_length):
    bleu_scores = []
    for key, val in encoding_test.items():
        image_features = val.reshape(1, 2048)
        generated_caption = greedy_search2(image_features, model, wordtoix, ixtoword, max_length)
        generated_caption = generated_caption.split()
        
        actual_caption = result_dict[key]
        references = [actual_caption.split()]
        
        bleu_score = calculate_bleu_score(references, generated_caption)
        bleu_scores.append(bleu_score)
        
    return bleu_scores

# Function to perform greedy search
def greedy_search2(photo, model, wordtoix, ixtoword, max_length):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo, sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

# Calculate BLEU scores for test dataset
bleu_scores = get_bleu_scores(encoding_test, result_dict, model, wordtoix, ixtoword, max_length)
average_bleu_score = np.mean(bleu_scores)
print("Average BLEU score:", average_bleu_score)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Average BLEU score: 0.21169096701589502
